In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_from_disk, list_datasets, load_dataset, disable_caching, load_metric
import pandas as pd
import ast
import matplotlib.pyplot as plt

In [2]:
def blue_score(results, tokenizer):
    predictions = []
    references = []

    with torch.no_grad():
        for i in range(len(results)):
            output_ids = ast.literal_eval(results["output_ids"][i])
            target_ids = ast.literal_eval(results["target_ids"][i])
        
            prediction = tokenizer.decode(output_ids, skip_special_tokens=True)
            reference = tokenizer.decode(target_ids, skip_special_tokens=True)
            
            predictions.append(prediction)
            references.append([reference])
    bleu = load_metric("sacrebleu")
    bleu.add_batch(predictions=predictions, references=references)
    return bleu.compute()["score"]


def compute_fnr(results):
    count = 0
    z_list = []    
    return (results["z_score"] < 4).sum()/len(results)

def compute_fpr(results):
    count = 0
    z_list = []    
    return (results["z_score"] >= 4).sum()/len(results)

In [3]:
MODEL_NAME="facebook/nllb-200-3.3B"
#model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, cache_dir="/data1/takezawa/huggingface", torch_dtype=torch.float16, revision="1a07f7d195896b2114afcb79b7b57ab512e7b43e")

In [4]:
language = "de-en"
gamma = 0.0001

# texts generated by the NS-Watermark
results_ns_watermark = pd.read_csv(f"results/wmt_{language}/z_score/ns_watermark_{gamma}_1.csv.gz")

# texts written by human
results_human = pd.read_csv(f"results/wmt_{language}/z_score/human_{gamma}.csv.gz")

In [5]:
if language in ["en-fr","en-de"]:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir="/data1/takezawa/huggingface", revision="1a07f7d195896b2114afcb79b7b57ab512e7b43e")
elif language == "de-en":
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, src_lang="deu_Latn", cache_dir="/data1/takezawa/huggingface", revision="1a07f7d195896b2114afcb79b7b57ab512e7b43e")
elif language == "fr-en":
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, src_lang="fra_Latn", cache_dir="/data1/takezawa/huggingface", revision="1a07f7d195896b2114afcb79b7b57ab512e7b43e")

In [6]:
print(f"NS-Watermark (gamma={gamma})")
print(f"\t BLEU={blue_score(results_ns_watermark, tokenizer)}, FNR={compute_fnr(results_ns_watermark)}")

print("Human")
print(f"\t FPR={compute_fpr(results_human)}")

NS-Watermark (gamma=0.0001)


/tmp/ipykernel_8856/1741476300.py:15: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu = load_metric("sacrebleu")


	 BLEU=38.85980449519435, FNR=0.0
Human
	 FPR=0.0
